In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

!nvidia-smi

/kaggle/input/elonmusk-2019-tweets/2019.csv
Tue Nov 28 15:08:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                            

In [ ]:
!wget -O prompts.csv https://raw.githubusercontent.com/krea-ai/open-prompts/main/data/1k.csv

In [ ]:
!ls

In [2]:
# Assuming the dataset is in the '/kaggle/input/' directory
file_path = '/kaggle/input/elonmusk-2019-tweets/2019.csv'

In [4]:
tweets = pd.read_csv(file_path)

In [5]:
tweets.head()

,Unnamed: 0,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,0,1211071324518531072,1210918835861241856,1.577577e+12,2019-12-28 23:47:59,0,NaN,@Joe__Wakefield @austinbarnard45 @tjq1190 @tyg...,en,[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'austinbarnard45', 'name': 'A...",NaN,NaN,NaN,NaN
1,1,1211069054779301894,1210918835861241856,1.577576e+12,2019-12-28 23:38:57,0,NaN,@austinbarnard45 @tjq1190 @tyger_cyber @fawful...,en,[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'austinbarnard45', 'name': 'A...",NaN,NaN,NaN,NaN
2,2,1211064937004589056,1210918835861241856,1.577575e+12,2019-12-28 23:22:36,0,NaN,@IrinaAntony @tjq1190 @tyger_cyber @fawfulfan ...,en,[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'IrinaAntony', 'name': '✨⭐️✨'...",NaN,NaN,NaN,NaN
3,3,1211054942192119808,1210918835861241856,1.577573e+12,2019-12-28 22:42:53,0,NaN,@tjq1190 @tyger_cyber @fawfulfan @_Mikemo He d...,en,[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'tjq1190', 'name': 'Tyler J. ...",NaN,NaN,NaN,NaN
4,4,1211051740562366464,1210774027054145536,1.577572e+12,2019-12-28 22:30:09,0,NaN,@geofficient Pretty much,en,[],...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'geofficient', 'name': 'Geoff...",NaN,NaN,NaN,NaN


In [7]:
tweets.columns

Index(['Unnamed: 0', 'id', 'conversation_id', 'created_at', 'date', 'timezone',
       'place', 'tweet', 'language', 'hashtags', 'cashtags', 'user_id',
       'user_id_str', 'username', 'name', 'day', 'hour', 'link', 'urls',
       'photos', 'video', 'thumbnail', 'retweet', 'nlikes', 'nreplies',
       'nretweets', 'quote_url', 'search', 'near', 'geo', 'source',
       'user_rt_id', 'user_rt', 'retweet_id', 'reply_to', 'retweet_date',
       'translate', 'trans_src', 'trans_dest'],
      dtype='object')

Majority of features are useless for use, thus we will just remove them, and will keep just 'tweet' feature.

In [8]:
tweets = tweets[['id', 'tweet']]

### Preprocessing

In [50]:
# Use regular expression to remove all the tagged user accounts as they donot contribute to the text
import re

def clean_tweet(tweet):
    # Remove mentions
    mention_pattern = r'@\w+'
    cleaned_tweet = re.sub(mention_pattern, '', tweet)

    # Remove extra spaces
    cleaned_tweet = ' '.join(cleaned_tweet.split())

    # Remove hyperlinks
    hyperlink_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    cleaned_tweet = re.sub(hyperlink_pattern, '', cleaned_tweet)

    return cleaned_tweet

In [51]:
tweets.isnull().sum()

id       0
tweet    0
dtype: int64

In [14]:
tweets.reset_index(inplace = True,drop = True)

In [58]:
tweets['tweet'] = tweets['tweet'].apply(clean_tweet)

In [59]:
tweets.head()

,id,tweet
0,1211071324518531072,This is a pretty awful lie. I left South Afric...
1,1211069054779301894,"This person blocked me, so can’t read the twee..."
2,1211064937004589056,We started Zip2 with ~$2k from me plus my over...
3,1211054942192119808,He didn’t own an emerald mine &amp; I worked m...
7,1211029491188948992,Hopefully working in Caribbean by end of 2020....


In [55]:
tweets = tweets.loc[tweets.tweet.str.split(' ').str.len() > 10]

In [56]:
tweets.shape

(4568, 2)

In [32]:
! pip install aitextgen -q

In [33]:
model="EleutherAI/gpt-neo-125M"

In [ ]:
prompts_gt6 = prompts_gt6.drop('raw_data', axis = 1).reset_index(drop=True)

In [57]:
tweets.to_csv("input_text_cleaned.txt", columns=["tweet"], header=False, index=False)


In [34]:
from aitextgen.TokenDataset import TokenDataset

In [36]:
data = TokenDataset('./input_text_cleaned.txt', line_by_line=True)

  0%|          | 0/4568 [00:00<?, ?it/s]

In [37]:
from aitextgen import aitextgen

In [40]:
ai = aitextgen(tf_gpt2="124M",  to_gpu=True)
# ai = aitextgen(model="EleutherAI/gpt-neo-125M", to_gpu=True)

Fetching checkpoint:   0%|          | 0.00/77.0 [00:00<?, ?it/s]

Fetching hparams.json:   0%|          | 0.00/90.0 [00:00<?, ?it/s]

Fetching model.ckpt.data-00000-of-00001:   0%|          | 0.00/498M [00:00<?, ?it/s]

Fetching model.ckpt.index:   0%|          | 0.00/5.21k [00:00<?, ?it/s]

Fetching model.ckpt.meta:   0%|          | 0.00/471k [00:00<?, ?it/s]

Converting TensorFlow checkpoint from /kaggle/working/aitextgen/124M
Loading TF weight model/h0/attn/c_attn/b with shape [2304]
Loading TF weight model/h0/attn/c_attn/w with shape [1, 768, 2304]
Loading TF weight model/h0/attn/c_proj/b with shape [768]
Loading TF weight model/h0/attn/c_proj/w with shape [1, 768, 768]
Loading TF weight model/h0/ln_1/b with shape [768]
Loading TF weight model/h0/ln_1/g with shape [768]
Loading TF weight model/h0/ln_2/b with shape [768]
Loading TF weight model/h0/ln_2/g with shape [768]
Loading TF weight model/h0/mlp/c_fc/b with shape [3072]
Loading TF weight model/h0/mlp/c_fc/w with shape [1, 768, 3072]
Loading TF weight model/h0/mlp/c_proj/b with shape [768]
Loading TF weight model/h0/mlp/c_proj/w with shape [1, 3072, 768]
Loading TF weight model/h1/attn/c_attn/b with shape [2304]
Loading TF weight model/h1/attn/c_attn/w with shape [1, 768, 2304]
Loading TF weight model/h1/attn/c_proj/b with shape [768]
Loading TF weight model/h1/attn/c_proj/w with shap

Save PyTorch model to aitextgen/pytorch_model.bin
Save configuration file to aitextgen/config.json


In [ ]:
# How can we try the performance of model before finetuning

In [42]:
ai.train('input_text_cleaned.txt',
         line_by_line=True,
         from_cache=False,
         num_steps=500,
         generate_every=100,
         save_every=500,
         save_gdrive=False,
         learning_rate=1e-3,
         fp16=False,
         batch_size=1, 
         )

  0%|          | 0/4568 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:448: LightningDeprecationWarning: Setting `Trainer(gpus=-1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=-1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/configuration_validator.py:259: LightningDeprecationWarning: The `Callback.on_batch_end` hook was deprecated in v1.6 and will be removed in v1.8. Please use `Callback.on_train_batch_end` instead.
  f"The `Callback.{hook}` hook was deprecated in v1.6 and"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/500 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [43]:
ai.save()


In [44]:
prompt_ai = aitextgen(model_folder = '.', to_gpu=True)

In [47]:
print(prompt_ai.generate(prompt = "We have to "))

We have to  Falcon Raptor SN-1.0 on SN-1. We are essentially a steward of life &amp; duty-bound imo to ensure its continuance.  http://t.co/R1d6Zw9hH

None


In [66]:
print(prompt_ai.generate(prompt = "Donald Trump can "))

Donald Trump can  anyplace in the world &amp; I will do some great terms with the Air Force. Major pucker factor for Space Station.

None


In [62]:
print(prompt_ai.generate(prompt = "Tesla will "))

Tesla will  charge a cool price for the 75 kWh pack pack &amp; have a fixed price for the pack

None


In [67]:
ai.save_for_upload('sd-prompt-generator-gpt-neo')

tokenizer config file saved in sd-prompt-generator-gpt-neo/tokenizer_config.json
Special tokens file saved in sd-prompt-generator-gpt-neo/special_tokens_map.json
